In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, welch, find_peaks, medfilt, windows, spectrogram, detrend,  iirnotch

# -- 1. Decode raw BIN → ADC cube () --
def decode_iwr6843_data(filename,
                        num_rx=4,
                        num_adc_samples=256,
                        num_chirps=128,
                        header_bytes=0):
    with open(filename, 'rb') as f:
        f.seek(header_bytes)
        raw = np.fromfile(f, dtype=np.int16)
    raw = raw.reshape(-1, 2)
    complex_data = raw[:,0] + 1j*raw[:,1]
    samp_pf = num_rx * num_chirps * num_adc_samples
    num_frames = len(complex_data) // samp_pf

    adc = np.empty((num_frames, num_rx, num_chirps, num_adc_samples),
                   dtype=complex)
    for fr in range(num_frames):
        for rx in range(num_rx):
            st = fr*samp_pf + rx*(num_chirps*num_adc_samples)
            en = st + (num_chirps*num_adc_samples)
            adc[fr,rx] = complex_data[st:en].reshape(num_chirps, num_adc_samples)
    return adc

# -- 2. Range FFT --
def compute_range_profiles(adc):
    return np.fft.fft(adc, axis=-1)

# -- 3. Range–Angle map --
def compute_range_angle_map(rp, n_angle_bins=256):
    avg_rx = rp.mean(axis=(0,2))
    ang = np.fft.fftshift(
        np.fft.fft(avg_rx, n=n_angle_bins, axis=0),
        axes=0
    )
    return np.abs(ang).T

# -- 4. Physical‐distance‐based target pick --
def detect_targets_physical(ra_map, num_targets=2,
                            min_dist_m=0.3, B=3.9e9,
                            n_angle_bins=256, R_mean=4.0):
    c = 3e8
    rng_res = c/(2*B)
    min_rb = int(np.ceil(min_dist_m/rng_res))
    ang_res = 180.0/n_angle_bins
    ang_sep = np.degrees(np.arcsin(min_dist_m/R_mean))
    min_ab = int(np.ceil(ang_sep/ang_res))

    R, A = ra_map.shape
    idxs = np.argsort(ra_map.flatten())[::-1]
    targets = []
    for idx in idxs:
        r, a = divmod(idx, A)
        if r < min_rb or r > R - min_rb:
            continue
        if any(abs(r-r0)<min_rb and abs(a-a0)<min_ab for r0,a0 in targets):
            continue
        targets.append((r,a))
        if len(targets) == num_targets:
            break
    return targets

# -- 5. MVDR beamforming ±3‐bin slab --
def separate_with_mvdr(adc, targets,
                       n_angle_bins=256,
                       fc=60e9,
                       rng_win=3,
                       eps=1e-6):
    frames, num_rx, chirps, samples = adc.shape
    rp = np.fft.fft(adc, axis=-1)
    rng_bins = rp.shape[-1]
    angle_axis = np.linspace(-90, 90, n_angle_bins)

    def steer(theta):
        lam = 3e8/fc
        d = lam/2
        k = 2*np.pi/lam
        idx = np.arange(num_rx)
        a = np.exp(-1j*k*d*idx*np.sin(np.deg2rad(theta)))
        return a[:,None]

    beams = np.zeros((frames, len(targets)), dtype=complex)
    for i, (rbin, abin) in enumerate(targets):
        lo = max(0, rbin-rng_win)
        hi = min(rng_bins, rbin+rng_win+1)
        gated = rp[:,:,:,lo:hi]
        X = gated.transpose(1,0,2,3).reshape(num_rx, -1)
        Rcov = X @ X.conj().T / X.shape[1] + eps*np.eye(num_rx)
        a_vec = steer(angle_axis[abin])
        w = np.linalg.inv(Rcov) @ a_vec
        w /= (a_vec.conj().T @ np.linalg.inv(Rcov) @ a_vec)
        for fr in range(frames):
            Y = gated[fr].reshape(num_rx, -1)
            bf = (w.conj().T @ Y).reshape(-1)
            beams[fr, i] = bf.sum()
    return beams

# -- 6. Filters & HR extraction --
def bandpass_filter(x, low, high, fs, order=4):
    b, a = butter(order, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, x)

def estimate_rate_welch(x, fs, fmin, fmax):
    f, P = welch(x, fs=fs, nperseg=min(len(x), int(5*fs)))
    mask = (f>=fmin) & (f<=fmax)
    return (f[mask][np.argmax(P[mask])] * 60.0) if np.any(mask) else np.nan
FILE = "plots/SubsetD/084_TopFront_2m_45deg_Man2_CondA_CondG_ADC256_Chirp128_SR1000_RE60_FR40_Gain30_FS30_IWR_0.bin"
adc   = decode_iwr6843_data(FILE)
print("1) ADC data shape:", adc.shape)

# 2) Range profiles → RA map → target detection → beamforming
rp     = compute_range_profiles(adc)
ra_map = compute_range_angle_map(rp, n_angle_bins=256)
targets= detect_targets_physical(ra_map, num_targets=2, min_dist_m=0.8,
                                 B=3.9e9, n_angle_bins=256, R_mean=4.0)
beams  = separate_with_mvdr(adc, targets, n_angle_bins=256, rng_win=3)
print("5) Beamformed shape:", beams.shape)

# 3) Windowed HR
FS         = 40.0         # frames per second
WINDOW_SEC = 10
window_sz  = int(WINDOW_SEC * FS)

all_hr_means = []

for beam_idx in range(beams.shape[1]):
    z     = beams[:, beam_idx]
    phase = np.unwrap(np.angle(z))
    dphi  = np.gradient(phase) * FS / (2*np.pi)

    hr_list = []

    n_wins = len(z) // window_sz
    for w in range(n_wins):
        seg = dphi[w*window_sz : (w+1)*window_sz]

        # — HR via Welch —
        heart_w = bandpass_filter(seg, 0.8, 3.0, FS)
        hr_w    = estimate_rate_welch(heart_w, FS, 0.8, 3.0)

        # — discard windows with implausible HR or low SNR —
        if not np.isnan(hr_w) and 40.0 <= hr_w <= 200.0:
            hr_list.append(hr_w)

    # mean over valid windows
    hr_mean = np.mean(hr_list) if hr_list else np.nan
    all_hr_means.append(hr_mean)

    print(f"Patient {beam_idx+1}: ⌀HR = {hr_mean:.1f} bpm")

def hr_fn(_z, _fs, val=hr_mean):
        return val

def estimate_rr_ridge_notch(z, fs,
                            rr_band=(0.05,0.5),
                            win_sec=15,
                            step_sec=2,
                            hr_estimator=None,
                            notch_Q=30):
    # 1) instantaneous frequency & detrend
    phase = np.unwrap(np.angle(z))
    dphi  = np.gradient(phase) * fs/(2*np.pi)
    dphi  = detrend(dphi)

    # 2) notch out heart if available
    if hr_estimator is not None:
        hr_bpm = hr_estimator(z, fs)
        if np.isfinite(hr_bpm):
            f0 = hr_bpm/60.0
            b,a = iirnotch(f0/(fs/2), Q=notch_Q)
            dphi = filtfilt(b, a, dphi)

    # 3) spectrogram with robust segment/overlap sizing
    #    ensure nperseg <= len(dphi) and noverlap < nperseg
    nperseg0  = int(win_sec * fs)
    nperseg   = min(nperseg0, len(dphi))
    noverlap0 = int((win_sec - step_sec) * fs)
    noverlap  = min(noverlap0, nperseg-1)
    if nperseg < 2:
        return np.nan

    f, t, Sxx = spectrogram(
        dphi, fs=fs,
        nperseg=nperseg,
        noverlap=noverlap,
        nfft=4096,
        scaling='density',
        mode='psd'
    )

    # 4) pick ridge in respiratory band
    mask   = (f>=rr_band[0]) & (f<=rr_band[1])
    if not np.any(mask):
        return np.nan

    S_band = Sxx[mask,:]
    f_band = f[mask]
    idx    = np.argmax(S_band, axis=0)
    ridge  = f_band[idx] * 60.0
    rr_med = np.median(ridge)
    return float(rr_med) if 5.0 <= rr_med <= 60.0 else np.nan

for i in range(beams.shape[1]):
    z    = beams[:,i]
    rr2  = estimate_rr_ridge_notch(
               z, FS,
               rr_band=(0.05,0.5),
               win_sec=15,
               step_sec=2,
               hr_estimator=hr_fn,  # your existing HR fcn
               notch_Q=30
           )
    print(f"Patient #{i+1}: RR_notch = {rr2:.1f} bpm")


# CONDITIONS A AND G
# RR: 14 AND 30
# HR: 80 AND 120

1) ADC data shape: (1410, 4, 128, 256)
5) Beamformed shape: (1410, 2)
Patient 1: ⌀HR = 140.0 bpm
Patient 2: ⌀HR = 92.0 bpm
Patient #1: RR_notch = 6.4 bpm
Patient #2: RR_notch = 16.4 bpm


In [4]:

"""
Three-bed vital-sign extraction with ONE IWR6843AOP sensor.

Pipeline:
1) Range FFT per frame
2) Gate near 3.50 m (bed B) and ~3.536 m (beds A/C)
3) MUSIC/MVDR AoA at the side gate to split A vs C
4) MVDR beamform to A, B, C -> complex slow-time signals s_A[t], s_B[t], s_C[t]
5) Phase-unwrapping -> displacement
6) Band-pass filters & Welch PSD -> RR (breaths/min) and HR (beats/min)\
"""

import os
import numpy as np
import numpy.linalg as LA
import scipy.signal as sig
import matplotlib.pyplot as plt
from dataclasses import dataclass

# -----------------------------
# Radar configuration (edit if needed)
# -----------------------------
@dataclass
class RadarParams:
    fc_hz: float = 60.25e9          # start/center frequency [Hz]
    slope_hz_per_s: float = 60e12   # 60 MHz/us
    fs_adc: float = 10e6            # ADC sample rate [Hz]
    num_samples: int = 256          # ADC samples per chirp
    num_chirps: int = 128           # chirps per frame (TDM across 3 Tx)
    frame_period_s: float = 0.040   # 40 ms per frame
    c: float = 299_792_458.0        # speed of light

    @property
    def wavelength(self):
        return self.c / self.fc_hz

    @property
    def chirp_duration(self):
        # If you want: B = slope * T; with B <= fs_adc * num_samples / decim
        # In lab we usually set T independently; not strictly needed here.
        # Keep for reference:
        return 80e-6  # seconds (typical in your setup)

    @property
    def bandwidth(self):
        return self.slope_hz_per_s * self.chirp_duration

    @property
    def range_resolution(self):
        return self.c / (2 * self.bandwidth)

    @property
    def rmax_unambiguous(self):
        # Rmax = c * fs / (2 * slope)
        return self.c * self.fs_adc / (2 * self.slope_hz_per_s)

    @property
    def slowtime_fs(self):
        # frames per second
        return 1.0 / self.frame_period_s

# -----------------------------
# AOP virtual array geometry (meters)
# -----------------------------
def aop_virtual_coords_m():
    # Rx in mm -> m
    Rx = np.array([
        [ 0.0,  +1.8, 0.0],
        [+1.8,   0.0, 0.0],
        [ 0.0,  -1.8, 0.0],
        [-1.8,   0.0, 0.0],
    ]) * 1e-3
    Tx = np.array([
        [+0.9, +0.9, 0.0],
        [-0.9, +0.9, 0.0],
        [ 0.0, -0.9, 0.0],
    ]) * 1e-3
    virt = []
    for t in Tx:
        for r in Rx:
            virt.append(t + r)
    virt = np.array(virt)  # (12,3)
    # We'll use x (left-right) primarily; y exists but we scan only azimuth.
    return virt

# -----------------------------
# Utilities
# -----------------------------
def steering_az(az_deg, coords_xyz, wavelength):
    """
    1D azimuth steering using only x-axis projection (left-right across beds).
    Assumes small elevation variation (targets below sensor, same height).
    """
    az = np.deg2rad(az_deg)
    phase = (2*np.pi/wavelength) * (coords_xyz[:,0] * np.sin(az))
    return np.exp(1j * phase)

def mvdr_spectrum(R, coords, wavelength, az_grid_deg):
    R_inv = LA.pinv(R + 1e-6*np.eye(R.shape[0]))
    P = np.zeros_like(az_grid_deg, dtype=float)
    for i, az in enumerate(az_grid_deg):
        a = steering_az(az, coords, wavelength)[:,None]
        denom = np.real(a.conj().T @ R_inv @ a)[0,0]
        P[i] = 1.0 / max(denom, 1e-12)
    return P / P.max()

def music_spectrum(R, coords, wavelength, az_grid_deg, n_sources=2):
    # Eigen-decomposition
    evals, evecs = LA.eigh(R)
    idx = np.argsort(evals)[::-1]
    evecs = evecs[:, idx]
    En = evecs[:, n_sources:]  # noise subspace
    P = np.zeros_like(az_grid_deg, dtype=float)
    for i, az in enumerate(az_grid_deg):
        a = steering_az(az, coords, wavelength)
        denom = LA.norm(En.conj().T @ a)**2
        P[i] = 1.0 / max(denom, 1e-12)
    return P / P.max()

def find_two_peaks(x, x_axis, min_sep_deg=4.0):
    idx = sig.find_peaks(x, distance=int(min_sep_deg / (x_axis[1]-x_axis[0])))[0]
    if len(idx) < 2:
        # fallback: take top-2 indices
        idx = np.argsort(x)[-2:]
    idx = idx[np.argsort(x[idx])][::-1]  # sort by height desc
    p1, p2 = idx[0], idx[1]
    return (x_axis[p1], x[p1]), (x_axis[p2], x[p2])

def hann_win(n):
    return np.hanning(n)

def db10(x):
    return 10*np.log10(np.maximum(x, 1e-12))

# -----------------------------
# Range processing
# -----------------------------
def range_fft(cube12_chirps_samples, rp: RadarParams, nfft=1024):
    """
    cube: (12, num_chirps, num_samples) complex
    returns:
      Xrng: (12, num_chirps, nfft) complex spectra (0..N-1)
      range_axis: (nfft,) meters
      range_profile_sum: (nfft,) power integrated over ants+chirps
    """
    W = hann_win(rp.num_samples)[None,None,:]
    x = cube12_chirps_samples * W
    X = np.fft.rfft(x, n=nfft, axis=2)  # one-sided is fine
    # Frequency axis (beat frequency)
    k = np.arange(X.shape[2])
    f = (k / nfft) * rp.fs_adc
    rng = rp.c * f / (2 * rp.slope_hz_per_s)
    rpwr = np.mean(np.abs(X)**2, axis=(0,1))
    return X, rng, rpwr

# -----------------------------
# Angle processing helpers
# -----------------------------
def snapshot_cov_at_range(Xrng, rng_bin):
    """
    From range-FFT cube -> covariance across 12 virtual antennas using chirp snapshots.
    Xrng: (12, num_chirps, nfft)
    rng_bin: int
    returns R (12x12), and per-chirp snapshots S (12 x num_chirps)
    """
    S = Xrng[:,:,rng_bin]  # (12, num_chirps)
    R = (S @ S.conj().T) / S.shape[1]
    return R, S

def mvdr_weights(R, a):
    R_inv = LA.pinv(R + 1e-6*np.eye(R.shape[0]))
    w = (R_inv @ a) / (a.conj().T @ R_inv @ a)
    return w

# -----------------------------
# Vital sign extraction
# -----------------------------
def unwrap_phase_to_disp(z_t, wavelength):
    phi = np.unwrap(np.angle(z_t))
    disp = (wavelength / (4*np.pi)) * phi
    return disp

def bandpass(x, fs, f1, f2, order=4):
    b, a = sig.butter(order, [f1/(fs/2), f2/(fs/2)], btype='band')
    return sig.filtfilt(b, a, x)

def welch_peak_freq(x, fs, f_lo, f_hi):
    nperseg = min(len(x), max(128, int(fs*8)))
    f, P = sig.welch(x, fs=fs, nperseg=nperseg, noverlap=nperseg//2)
    m = (f >= f_lo) & (f <= f_hi)
    if not np.any(m):
        return np.nan, np.nan, (f, P)
    idx = np.argmax(P[m])
    fpk = f[m][idx]
    ppk = P[m][idx]
    # Simple SNR metric vs. band median
    snr = 10*np.log10(ppk / (np.median(P[m]) + 1e-12))
    return fpk, snr, (f, P)

# -----------------------------
# Main routine
# -----------------------------
class ThreeBedVitals:
    def __init__(self, rp: RadarParams):
        self.rp = rp
        self.coords = aop_virtual_coords_m()

    def detect_range_bins(self, rpwr, rng_axis, approx_center_m=3.50, approx_side_m=None):
        if approx_side_m is None:
            approx_side_m = np.sqrt(3.5**2 + 0.5**2)

        def pick_near(target_m, window_m=0.06):
            mask = (rng_axis > target_m - window_m/2) & (rng_axis < target_m + window_m/2)
            idx = np.argmax(rpwr[mask])
            return np.where(mask)[0][0] + idx

        idx_center = pick_near(approx_center_m, 0.08)
        idx_side   = pick_near(approx_side_m, 0.08)
        return idx_center, idx_side

    def estimate_side_aoas(self, Xrng_accum, rng_bin_side, method="mvdr"):
        # Accumulate covariance over a few frames for stability
        R = np.zeros((12,12), dtype=complex)
        nsnaps = 0
        for Xrng in Xrng_accum:
            Rf, _ = snapshot_cov_at_range(Xrng, rng_bin_side)
            R += Rf
            nsnaps += 1
        R /= max(nsnaps,1)

        az_grid = np.linspace(-30, 30, 721)  # 0.083° step
        if method.lower() == "music":
            P = music_spectrum(R, self.coords, self.rp.wavelength, az_grid, n_sources=2)
        else:
            P = mvdr_spectrum(R, self.coords, self.rp.wavelength, az_grid)
        (az1, _), (az2, _) = find_two_peaks(P, az_grid, min_sep_deg=4.0)
        # Assign left/right
        azA = min(az1, az2)
        azC = max(az1, az2)
        return (azA, azC), (az_grid, P)

    def beamform_frame_sample(self, Xrng, rng_bin, az_deg):
        R, S = snapshot_cov_at_range(Xrng, rng_bin)
        a = steering_az(az_deg, self.coords, self.rp.wavelength)[:,None]
        w = mvdr_weights(R, a)
        # Average across chirps for the frame snapshot
        z = (w.conj().T @ S).mean(axis=1).ravel()
        return z  # complex scalar

    def run(self, cube_frames_12_chirps_samples, nfft_range=1024,
            approx_center_m=3.50, approx_side_m=None, use_music=False,
            show_plots=True):

        rp = self.rp
        F, V, C, S = cube_frames_12_chirps_samples.shape
        assert V == 12, "Expecting 12 virtual channels."

        # 1) Range FFT per frame + pick range gates using first few frames
        Xrng_list, rpwr_list = [], []
        for f in range(F):
            Xrng, rng_axis, rpwr = range_fft(cube_frames_12_chirps_samples[f], rp, nfft=nfft_range)
            Xrng_list.append(Xrng)
            rpwr_list.append(rpwr)
        rpwr_mean = np.mean(np.vstack(rpwr_list), axis=0)
        idx_center, idx_side = self.detect_range_bins(rpwr_mean, rng_axis, approx_center_m, approx_side_m)
        rng_center = rng_axis[idx_center]; rng_side = rng_axis[idx_side]

        # 2) AoA for side gate (accumulate first N frames)
        N_accum = min(20, F)
        (azA, azC), (az_grid, P_side) = self.estimate_side_aoas(
            Xrng_list[:N_accum], idx_side, method="music" if use_music else "mvdr"
        )
        # Bed B is near boresight; you can also estimate it:
        Rmid, _ = snapshot_cov_at_range(Xrng_list[0], idx_center)
        Pmid = mvdr_spectrum(Rmid, self.coords, rp.wavelength, az_grid)
        azB = az_grid[np.argmax(Pmid)]

        # 3) Beamform each frame at fixed AoAs to get complex slow-time
        zA, zB, zC = np.zeros(F, complex), np.zeros(F, complex), np.zeros(F, complex)
        for f in range(F):
            zA[f] = self.beamform_frame_sample(Xrng_list[f], idx_side,   azA)
            zC[f] = self.beamform_frame_sample(Xrng_list[f], idx_side,   azC)
            zB[f] = self.beamform_frame_sample(Xrng_list[f], idx_center, azB)

        # 4) Phase -> displacement (slow-time sampling = 1/frame_period)
        fs_slow = rp.slowtime_fs
        dA = unwrap_phase_to_disp(zA, rp.wavelength)
        dB = unwrap_phase_to_disp(zB, rp.wavelength)
        dC = unwrap_phase_to_disp(zC, rp.wavelength)

        # Detrend small DC drift
        dA = sig.detrend(dA, type='constant')
        dB = sig.detrend(dB, type='constant')
        dC = sig.detrend(dC, type='constant')

        # 5) Respiration & Heart bands
        respA = bandpass(dA, fs_slow, 0.10, 0.60)
        respB = bandpass(dB, fs_slow, 0.10, 0.60)
        respC = bandpass(dC, fs_slow, 0.10, 0.60)

        heartA = bandpass(dA, fs_slow, 0.80, 3.00)
        heartB = bandpass(dB, fs_slow, 0.80, 3.00)
        heartC = bandpass(dC, fs_slow, 0.80, 3.00)

        # 6) Welch peaks & quality (SNR in dB)
        fRA, snrRA, _ = welch_peak_freq(respA, fs_slow, 0.10, 0.60)
        fRB, snrRB, _ = welch_peak_freq(respB, fs_slow, 0.10, 0.60)
        fRC, snrRC, _ = welch_peak_freq(respC, fs_slow, 0.10, 0.60)

        fHA, snrHA, _ = welch_peak_freq(heartA, fs_slow, 0.80, 3.00)
        fHB, snrHB, _ = welch_peak_freq(heartB, fs_slow, 0.80, 3.00)
        fHC, snrHC, _ = welch_peak_freq(heartC, fs_slow, 0.80, 3.00)

        rrA = 60.0 * fRA; rrB = 60.0 * fRB; rrC = 60.0 * fRC
        hrA = 60.0 * fHA; hrB = 60.0 * fHB; hrC = 60.0 * fHC

        results = {
            "angles_deg": {"A": azA, "B": azB, "C": azC},
            "ranges_m":   {"center": float(rng_center), "side": float(rng_side)},
            "RR_bpm":     {"A": float(rrA), "B": float(rrB), "C": float(rrC)},
            "RR_SNR_dB":  {"A": float(snrRA), "B": float(snrRB), "C": float(snrRC)},
            "HR_bpm":     {"A": float(hrA), "B": float(hrB), "C": float(hrC)},
            "HR_SNR_dB":  {"A": float(snrHA), "B": float(snrHB), "C": float(snrHC)},
            "slowtime_fs": fs_slow,
            "series": {
                "disp_A": dA, "disp_B": dB, "disp_C": dC,
                "resp_A": respA, "resp_B": respB, "resp_C": respC,
                "heart_A": heartA, "heart_B": heartB, "heart_C": heartC
            },
            "diagnostics": {
                "az_grid": az_grid, "P_side": P_side, "P_mid": Pmid,
                "range_axis": rng_axis, "range_profile_mean": rpwr_mean
            }
        }

        if show_plots:
            self._plots(results)

        return results

    def _plots(self, res):
        rp = self.rp
        t = np.arange(len(res["series"]["disp_A"])) / rp.slowtime_fs

        # Range profile
        rng = res["diagnostics"]["range_axis"]
        rpwr = res["diagnostics"]["range_profile_mean"]
        plt.figure()
        plt.plot(rng, db10(rpwr))
        plt.axvline(res["ranges_m"]["center"], ls='--')
        plt.axvline(res["ranges_m"]["side"], ls='--')
        plt.xlabel("Range (m)"); plt.ylabel("Power (dB)")
        plt.title("Average range profile & gates"); plt.grid(True)

        # Side AoA spectrum
        plt.figure()
        azg = res["diagnostics"]["az_grid"]; Pside = res["diagnostics"]["P_side"]
        plt.plot(azg, Pside)
        plt.axvline(res["angles_deg"]["A"], ls='--')
        plt.axvline(res["angles_deg"]["C"], ls='--')
        plt.xlabel("Azimuth (deg)"); plt.ylabel("Normalized Spectrum")
        plt.title("Side gate AoA (MVDR/MUSIC)"); plt.grid(True)

        # Displacement traces
        plt.figure()
        plt.plot(t, res["series"]["disp_A"], label='A')
        plt.plot(t, res["series"]["disp_B"], label='B')
        plt.plot(t, res["series"]["disp_C"], label='C')
        plt.xlabel("Time (s)"); plt.ylabel("Displacement (m)")
        plt.title("Beamformed chest displacement (slow-time)"); plt.grid(True); plt.legend()

        # Respiration filtered
        plt.figure()
        plt.plot(t, res["series"]["resp_A"], label=f'A RR≈{res["RR_bpm"]["A"]:.1f} bpm')
        plt.plot(t, res["series"]["resp_B"], label=f'B RR≈{res["RR_bpm"]["B"]:.1f} bpm')
        plt.plot(t, res["series"]["resp_C"], label=f'C RR≈{res["RR_bpm"]["C"]:.1f} bpm')
        plt.xlabel("Time (s)"); plt.ylabel("Respiratory disp (m)")
        plt.title("Respiration-band"); plt.grid(True); plt.legend()

        # Heart filtered
        plt.figure()
        plt.plot(t, res["series"]["heart_A"], label=f'A HR≈{res["HR_bpm"]["A"]:.0f} bpm')
        plt.plot(t, res["series"]["heart_B"], label=f'B HR≈{res["HR_bpm"]["B"]:.0f} bpm')
        plt.plot(t, res["series"]["heart_C"], label=f'C HR≈{res["HR_bpm"]["C"]:.0f} bpm')
        plt.xlabel("Time (s)"); plt.ylabel("Cardiac disp (m)")
        plt.title("Heartbeat-band"); plt.grid(True); plt.legend()
        plt.show()

# -----------------------------
# Data loading (replace with your own)
# -----------------------------
def load_cube_npy(path):
    """
    Expects shape (frames, 12, chirps, samples) complex64.
    """
    cube = np.load(path)
    if cube.ndim != 4 or cube.shape[1] != 12:
        raise ValueError("Expecting npy with shape (F,12,C,S) complex.")
    return cube

# Stub for custom loader from raw .bin if you have one already
def load_cube_from_bin_stub(path) -> np.ndarray:
    """
    Replace with your own reader. Should return complex array with shape (F,12,C,S).
    """
    raise NotImplementedError("Implement your DCA1000/IWR6843 reader here.")

# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    rp = RadarParams()

    # ---- Choose ONE loader ----
    # cube = load_cube_npy("your_cube_Fx12xCxS_complex.npy")
    # OR:
    # cube = load_cube_from_bin_stub("your_raw.bin")

    # For demonstration, create a dummy array to verify code runs end-to-end:
    # (Remove this and load your real cube)
    F, C, S = 300, rp.num_chirps, rp.num_samples
    np.random.seed(0)
    cube = (np.random.randn(F,12,C,S) + 1j*np.random.randn(F,12,C,S)).astype(np.complex64) * 1e-3

    tbv = ThreeBedVitals(rp)
    results = tbv.run(cube, nfft_range=1024, approx_center_m=3.50, approx_side_m=np.sqrt(3.5**2+0.5**2),
                      use_music=False, show_plots=False)

    # Print summary
    print("Angles (deg):", results["angles_deg"])
    print("Ranges (m):", results["ranges_m"])
    print("Respiration (bpm):", results["RR_bpm"], "SNR_dB:", results["RR_SNR_dB"])
    print("Heart (bpm):", results["HR_bpm"], "SNR_dB:", results["HR_SNR_dB"])


/opt/tljh/user/lib/python3.10/site-packages/numpy/fft/_pocketfft.py:70: ComplexWarning: Casting complex values to real discards the imaginary part
  r = pfi.execute(a, is_real, is_forward, fct)
/tmp/ipykernel_177433/769860273.py:287: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  zA[f] = self.beamform_frame_sample(Xrng_list[f], idx_side,   azA)
/tmp/ipykernel_177433/769860273.py:288: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  zC[f] = self.beamform_frame_sample(Xrng_list[f], idx_side,   azC)
/tmp/ipykernel_177433/769860273.py:289: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future

Angles (deg): {'A': 29.916666666666664, 'B': -21.25, 'C': 30.0}
Ranges (m): {'center': 3.537590039876302, 'side': 3.561987212565104}
Respiration (bpm): {'A': 7.5, 'B': 15.0, 'C': 7.5} SNR_dB: {'A': 1.5217723895724422, 'B': 1.145282297159719, 'C': 1.5216319547105417}
Heart (bpm): {'A': 97.5, 'B': 67.5, 'C': 97.5} SNR_dB: {'A': 5.403792231576302, 'B': 9.55383312397419, 'C': 5.406275682413169}
